# Moa prediction

- main strategy
    - overfit to private

## prediction flow

- make 5 models
    - simple NN with autoencoder
    - resnet
    - 2 headed NN??
    - RNN
    - TabNet
    
- staking all of them
- ensemble 6 models above

## prediction consider

- cv와 lb간의 성능 상관관계가 없었기 때문에 lb를 통해서 성능 검증
- 파라미터 튜닝은 oof 성능을 통해서 검증
- label smoothing??

# options

In [1]:
# seed, batch, epoch config

LABEL_SMOOTHING=0.001

SEEDS = [120,2524]
SPLITS = 7
BATCH_SIZE = 64
EPOCHS = 60

VARIANCE_THRESHOLD = True

ADD_NON_TRAIN = True
ADD_SEEDS = SEEDS
ADD_SPLITS = SPLITS
ADD_BATCH_SIZE = BATCH_SIZE

ADD_EPOCHS = EPOCHS

RUN_SNN = True
SNN_SEEDS = SEEDS + [42]
SNN_SPLITS = SPLITS
SNN_BATCH_SIZE = BATCH_SIZE
SNN_EPOCHS = EPOCHS

RUN_NN = True
NN_SEEDS = SEEDS 
NN_SPLITS = SPLITS
NN_BATCH_SIZE = BATCH_SIZE
NN_EPOCHS = EPOCHS

RUN_NN2 = True
NN2_SEEDS = SEEDS
NN2_SPLITS = SPLITS
NN2_BATCH_SIZE = BATCH_SIZE
NN2_EPOCHS = EPOCHS

RUN_RNN = True
RNN_SEEDS = SEEDS
RNN_SPLITS = SPLITS
RNN_BATCH_SIZE = 128
RNN_EPOCHS = EPOCHS

RUN_TABNET = True
TAB_SEEDS = SEEDS
TAB_SPLITS = SPLITS
TAB_BATCH_SIZE = BATCH_SIZE
TAB_EPOCHS = EPOCHS

RUN_STACKING = True
STK_SEEDS = SEEDS + [42]
STK_SPLITS = SPLITS 
STK_EPOCHS = EPOCHS
STK_BATCH_SIZE = BATCH_SIZE

RE_RUN_NN = True
RE_NN_SEEDS = SEEDS

# library

In [2]:
import pandas as pd
import numpy as np
import random
import os

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

import sys
# sys.path.append('../input/pytorchtabnet/tabnet-develop')
# from pytorch_tabnet import tab_network
# sys.path.append('../input/interactivestratification/iterative-stratification-master')
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [3]:
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

Processing /kaggle/input/iterative-stratification/iterative-stratification-master
  Created wheel for iterative-stratification: filename=iterative_stratification-0.1.6-py3-none-any.whl size=8401 sha256=8454e446ea0d9fa572dd962b6708afcb15309a91e07594ce5cb5eeb13c46c821
  Stored in directory: /root/.cache/pip/wheels/b8/47/3f/eb4af42d124f37d23d6f13a4c8bbc32c1d70140e6e1cecb4aa
Successfully built iterative-stratification
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'

!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

from pytorch_tabnet import tab_network

Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to /tmp/pip-install-e_4af28s/pytorch-tabnet
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-tabnet: filename=pytorch_tabnet-2.0.1-py3-none-any.whl size=30355 sha256=4e7b41a5b0fe28d715328668819ce8e5664ddf40fa105ab2640e3aecf69b082a
  Stored in directory: /tmp/pip-ephem-wheel-cache-th7b3x25/wheels/a6/8e/aa/6f5ef6a2e389c8b5f7ea1c74bbb03ece8773b03c2b8955c334
Successfully built pytorch-tabnet
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
from sklearn.metrics import log_loss
import torch

# 시드값 고정
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    torch.manual_seed(seed)  
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

# 아래의 함수들이 어디서 사용되는지 확인
# Loss 산정하는 함수인듯
def swish(x):
    return x * K.sigmoid(x)

def show_metrics(valid_preds, show_each=True):
    metrics = []
    for _target in valid_preds.columns:
      logloss = log_loss(train_targets_scored.iloc[:,1:].loc[:, _target], valid_preds.loc[:, _target])
      metrics.append(logloss)
      if show_each:
        print(f'column: {_target}, log loss: {logloss}')
    print(f'OOF Metric: {np.mean(metrics)}')

    return metrics

In [6]:
train_features = pd.read_csv(f'../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv(f'../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(f'../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv(f'../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv(f'../input/lish-moa/sample_submission.csv')

In [7]:
test_col = list(
    set(train_targets_scored.columns.tolist())\
    - set(['sig_id'])
)

temp_df = pd.DataFrame()
temp_df['col'] = ''
temp_df['value_0'] = 0
temp_df['ratio'] = 0.0

for col in test_col:
    value0 = dict(
        train_targets_scored[col].value_counts()
    )[0]
    
    temp_df = temp_df.append(
            {'col': col,
             'value_0': value0,
             'ratio': value0/len(train_targets_scored),
            },
            ignore_index = True
    )

In [8]:
# very imbalanced data
temp_df

,col,value_0,ratio
0,atm_kinase_inhibitor,23808,0.999748
1,bacterial_50s_ribosomal_subunit_inhibitor,23734,0.996641
2,glutamate_receptor_agonist,23740,0.996893
3,sigma_receptor_antagonist,23778,0.998488
4,mucolytic_agent,23766,0.997984
...,...,...,...
201,cc_chemokine_receptor_antagonist,23712,0.995717
202,carbonic_anhydrase_inhibitor,23778,0.998488
203,ubiquitin_specific_protease_inhibitor,23808,0.999748
204,rna_polymerase_inhibitor,23789,0.998950


In [9]:
# 유전, 세포와 관련된 피처들만 따로 변수저장
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [10]:
print(f'GENES: {len(GENES)}')
print(f'CELLS: {len(CELLS)}')

GENES: 772
CELLS: 100


# Feature engineering

- 분산 임계치(variance Threshold)
    - test셋을 사용한 것(leaky), 사용하지 않은 것의 분산임계값
    - 화자는 오버핏을 예상하고 테스트셋을 포함(leaky)
    - RNN을 제외한 모든 학습테스트에 적용
    - RNN에서는 non-leaky 분산 임계치 사용
    - 분산임계치란?
<br></br>
<br></br>
- PCA features
    - leaky 사용
    - 주성분이 많은 피처, 적은 피처 두가지를 사용  
<br></br>
<br></br>
- Statistical features
    - 'sum', 'mean', 'std', 'skew', 'kurt', 'median'
 <br></br>
<br></br>   
- Rank Gauss, Standard Scaler
    - input 피처의 분포를 아웃풋 때 정규화를 통해서 가우시안 분포를 따르도록 하는 것
  <br></br>
<br></br>    
- Non scored targets
    - ??

In [11]:
_X_train = train_features.copy()
_X_test = test_features.copy()
_y_train = train_targets_scored.copy()


## variance threshold
- 메인 아이디어
    - 분산이 작은 변수는 정보의 함유량이 낮다.
    - 각 변수에 대한 분산을 계산한 뒤 임계값을 기준으로 임계값보다 낮은 분산을 가진 변수를 제거한다
    - Make sure features have the same scale

In [12]:
# 분산 임계치 라이브러리
from sklearn.feature_selection import VarianceThreshold

In [13]:
if VARIANCE_THRESHOLD:
    # 유전, 세포 관련 컬럼들만 모두 추출
    vt_cols = _X_train.loc[:,GENES+CELLS].columns

    # 분산 임계치를 0.8로 설정
    # 임계점을 기준으로 어떻게 수치변환을 하는지?
    # 수치변환을 하는 것이 아니라 각 변수별 분산을 구한다음 분석가가 정한 임계점을 기준으로(threshold)
    # 그 값보다 분산이 적은 변수들을 제거하는 방식으로 변수를 선택한다
    vh = VarianceThreshold(0.8)
    train_trans = pd.DataFrame(vh.fit_transform(_X_train.loc[:,GENES+CELLS]))
    # 분산임계치의 get_support() : Get a mask, or integer index, of the features selected
    # 변수 선택: 분산임계치를 기준으로 변수선택
    # .get_support() 함수가 분산이 기준치 미달인 변수들을 selection해주는 함수
    vt_cols = vt_cols[vh.get_support()]

    # leaky - test셋까지 포함해서 변환
    vt_cols_leaky = _X_train.loc[:,GENES+CELLS].columns

    vh_leaky = VarianceThreshold(0.8)
    # leaky한 방법이 테스트셋을 행병합을 통해서 병합한 뒤 진행하는거군...
    # leaky, non-leaky의 train 데이터 변수가 다르게 나옴
    train_trans = pd.DataFrame(vh_leaky.fit_transform(pd.concat([_X_train.loc[:,GENES+CELLS], _X_test.loc[:,GENES+CELLS]])))
    vt_cols_leaky = vt_cols_leaky[vh_leaky.get_support()]


## categorical feature wrangling

In [14]:
# categorical feature pre-processing

# cp_type - 범주형 변수 cp 제거, trt_cp에 대한 예측을 해야함
def deal_with_cp_type(_X_train, _X_test):
    train_trt_index = _X_train.cp_type == 'trt_cp'
    test_trt_index = _X_test.cp_type == 'trt_cp'

    _X_train = _X_train[train_trt_index]
    _X_test = _X_test
    del _X_train['cp_type']
    del _X_test['cp_type']

    return _X_train, _X_test, train_trt_index, test_trt_index

_X_train, _X_test, train_trt_index, test_trt_index = deal_with_cp_type(_X_train, _X_test)

# cp_dose 더미 변수화
def deal_with_cp_dose(_X_train, _X_test):
    _X_train = pd.concat([_X_train, pd.get_dummies(_X_train.cp_dose)], axis=1)
    del _X_train['cp_dose']
    _X_test = pd.concat([_X_test, pd.get_dummies(_X_test.cp_dose)], axis=1)
    del _X_test['cp_dose']

    return _X_train, _X_test

_X_train, _X_test = deal_with_cp_dose(_X_train, _X_test)


# sig_id 피처 제거
def deal_with_sig_id(_X_train, _X_test):
    del _X_train['sig_id']
    del _X_test['sig_id']

    return _X_train, _X_test

_X_train, _X_test = deal_with_sig_id(_X_train, _X_test)


# 예측해야하는 cp_type index만 추출
def deal_with_y(_y_train, train_trt_index):
    _y_train = _y_train[train_trt_index]
    del _y_train['sig_id']

    return _y_train

_y_train = deal_with_y(_y_train, train_trt_index)

# I guess 'D1' should be eliminated but anyway, I was using it. 

BASE_COLS = ['cp_time', 'D1', 'D2']

_X_train_dae = _X_train.copy()
_X_test_dae = _X_test.copy()

_X_train = _X_train.reset_index()

# target non-score
- 해당 대회에서 제공해준 데이터(사용할지 여부는 선택사항)
- Additional (optional) binary MoA responses for the training data. These are not predicted nor scored.
- 해당 데이터를 어떻게 활용할지는 밑에서 설명

In [15]:
non_scored_ones = pd.DataFrame()
non_scored_ones['col_name'] = ''
non_scored_ones['item_counts'] = 0

for col in train_targets_nonscored.columns[1:]:

    item_counts = len(train_targets_nonscored[train_targets_nonscored[col] == 1])

    non_scored_ones = non_scored_ones.append({'col_name':col, 'item_counts':item_counts}, ignore_index=True)

non_scored_target_cols = non_scored_ones[non_scored_ones.item_counts > 10].col_name

y_non_train = train_targets_nonscored[non_scored_target_cols]
y_non_train = y_non_train[train_trt_index].reset_index(drop=True)

# PCA

In [16]:
from sklearn.decomposition import PCA


In [17]:
# 주성분 수는 50개로 설정
def yield_pca(_X_train, _X_test, prefix, decomp_cols, comp=50, random_state=42):
    # pca confg 할당
    pca = PCA(n_components=comp, random_state=random_state)
    print(pca.n_components)
    # 주성분 분석을 진행하기 위해서 train, test데이터 병합 후 주성분 분석
    # pca fitting
    pca.fit(pd.concat([_X_train.loc[:,decomp_cols], _X_test.loc[:,decomp_cols]]))
    # 유전과 관련된 변수 약 700개를 차원축소 진행: 주성분 갯수를 50개로 했을 때 누적 설명력이 66%밖에 안됨
    # 적어도 75% 이상이 될 수 있도록 주성분 갯수를 조정했어야하지 않을까? 주성분 100개일 때 72%
    _X_train_PCA = pca.transform(_X_train[decomp_cols])
    _X_test_PCA = pca.transform(_X_test[decomp_cols])

    _X_train_PCA = pd.DataFrame(_X_train_PCA, columns=[f'pca_{prefix}-{i}' for i in range(comp)])
    _X_test_PCA = pd.DataFrame(_X_test_PCA, columns=[f'pca_{prefix}-{i}' for i in range(comp)])

    return _X_train_PCA, _X_test_PCA

In [18]:
# 주성분을 여러개로 세팅한 뒤 만든 데이터셋에 대해서 리스트업
_X_train_G_PCA, _X_test_G_PCA = yield_pca(_X_train, _X_test, 'G', GENES, 90)
_X_train_C_PCA, _X_test_C_PCA = yield_pca(_X_train, _X_test, 'C', CELLS, 27)
# Dense를 만든 이유가 뭐지... -> 질문 리스트
_X_train_G_PCA_Dense, _X_test_G_PCA_Dense = yield_pca(_X_train, _X_test, 'Gd', GENES, 8)
_X_train_C_PCA_Dense, _X_test_C_PCA_Dense = yield_pca(_X_train, _X_test, 'Cd', CELLS, 1)

90
27
8
1


In [19]:
_X_train = pd.concat([_X_train, _X_train_G_PCA], axis=1)
_X_train = pd.concat([_X_train, _X_train_C_PCA], axis=1)
_X_train = pd.concat([_X_train, _X_train_G_PCA_Dense], axis=1)
_X_train = pd.concat([_X_train, _X_train_C_PCA_Dense], axis=1)

_X_test = pd.concat([_X_test, _X_test_G_PCA], axis=1)
_X_test = pd.concat([_X_test, _X_test_C_PCA], axis=1)
_X_test = pd.concat([_X_test, _X_test_G_PCA_Dense], axis=1)
_X_test = pd.concat([_X_test, _X_test_C_PCA_Dense], axis=1)

In [20]:
# 컬럼만 따로 변수화
PCA_G = [col for col in _X_train.columns if col.startswith('pca_G-')] \
+ [col for col in _X_train.columns if col.startswith('pca_Gd-')]
PCA_C = [col for col in _X_train.columns if col.startswith('pca_C-')] \
+ [col for col in _X_train.columns if col.startswith('pca_Cd-')]

# statistic feature

In [21]:
def make_feature(data, target_feats, create, prefix):
    target_data = data.loc[:,target_feats]
    # getattr 함수: 모든 변수에 대해서 통계값을 뽑고 싶을 때 getattr(data, method)를 입력해주면 됨
    invoker = getattr(target_data, create)
    return pd.DataFrame(invoker(axis=1), columns=[f'{prefix}_{create}'])

In [22]:
for method in ['sum', 'mean', 'std', 'skew', 'kurt', 'median']: # min max
    _X_train = pd.concat([_X_train, make_feature(_X_train, GENES+CELLS, method, 'gce')], axis = 1)
    _X_train = pd.concat([_X_train, make_feature(_X_train, GENES, method, 'ge')], axis = 1)

    _X_test = pd.concat([_X_test, make_feature(_X_test, GENES+CELLS, method, 'gce')], axis = 1)
    _X_test = pd.concat([_X_test, make_feature(_X_test, GENES, method, 'ge')], axis = 1)

In [23]:
# stat변수들 변수화
GC_EX = [col for col in _X_train.columns if col.startswith('gce_')]
G_EX = [col for col in _X_train.columns if col.startswith('ge_')] 

# 변수 스케일 및 사분위수 변환

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer

In [25]:
train_col_names = _X_train.columns
test_col_names = _X_test.columns

# 사분위수 변환할 변수들 리스트업
TRANSFORM_TARGET_COLS = BASE_COLS + GENES + CELLS + PCA_G + PCA_C + G_EX + GC_EX

In [26]:
# rank_guass 변환 방식: quantileTransformer - 분포를 정규분포곡선 형식으로 변환
# 지정된 분위수에 맞게 데이터를 변환하는 함수(n_quantiles: default: 1000)

# 스케일을하거나 변환 및 주성분 분석을 진행할 때는 train, test셋 모두 병합을 한 뒤 진행을 한다.

# 모든 변수들에 대해서 rank_gauss방법을 사용한 이유는??
def rank_gauss(_X_train, _X_test, cols=TRANSFORM_TARGET_COLS, random_state=42):
    qt = QuantileTransformer(random_state=random_state, output_distribution='normal')
    qt.fit(pd.concat([_X_train[cols], _X_test[cols]]))
    _X_train[cols] = qt.transform(_X_train[cols])
    _X_test[cols] = qt.transform(_X_test[cols])

    return _X_train, _X_test

# 표준정규분포 스케일러
def standard_scaler(_X_train, _X_test, cols=TRANSFORM_TARGET_COLS):
    ss = StandardScaler()
    ss.fit(pd.concat([_X_train[cols], _X_test[cols]]))
    _X_train[cols] = ss.transform(_X_train[cols])
    _X_test[cols] = ss.transform(_X_test[cols])

    return _X_train, _X_test


In [27]:
_X_train, _X_test = rank_gauss(_X_train, _X_test)
_X_train, _X_test = standard_scaler(_X_train, _X_test)

_X_train.columns = train_col_names
_X_test.columns = test_col_names

# 지금까지 만든 변수들을 이용해서 다시 분산임계치 변수선택

In [28]:
if VARIANCE_THRESHOLD:
    # 기존 분산임계치로 추출한 변수 + pca, stat으로 생성한 변수들을 이용하여 변수선택
    vt_cols = vt_cols.values.tolist() + BASE_COLS + PCA_G + PCA_C + GC_EX + G_EX

    _X_train_nl = pd.concat([_X_train.iloc[:,:1], _X_train.loc[:,vt_cols]], axis=1)
    _X_test_nl = _X_test.loc[:,vt_cols]
    
        
    vt_cols_leaky = vt_cols_leaky.values.tolist() + BASE_COLS + PCA_G + PCA_C + GC_EX + G_EX

    _X_train = pd.concat([_X_train.iloc[:,:1], _X_train.loc[:,vt_cols_leaky]], axis=1)
    _X_test = _X_test.loc[:,vt_cols_leaky]

    GENES = [col for col in _X_train.columns if col.startswith('g-')]
    CELLS = [col for col in _X_train.columns if col.startswith('c-')]

In [29]:
# set_index: 특정 컬럼을 인덱스로 옮기는 것
_X_train = _X_train.set_index('index')
X_train = _X_train.copy()
X_test = _X_test.copy()

# no leakage
_X_train_nl = _X_train_nl.set_index('index')
X_train_nl = _X_train_nl.copy()
X_test_nl = _X_test_nl.copy()

y_train = _y_train.copy()

# 모델링
- Non-scored target model
- simple NN
- RNN
- Resnet
- TabNet

## Non-scored target model
- non_target 관련 데이터에 대한 모델링이네?

In [30]:
def train_and_predict(
    name,
    model_func, 
    _X_i_train, 
    _y_i_train, 
    _X_i_test, 
    orig_targets,
    result_template,
    seeds, 
    splits, 
    epochs, 
    batch_size, 
    shuffle_rows=False, 
    pick_col_size=800, 
    do_show_metrics=True, 
    show_each_metrics=True
):
    
    st = time.time()

    # 리스트인지 확인하는 함수
    is_list = isinstance(_X_i_train, list)

    # train_targets_nonscored 데이터에서 필요한 데이터들만 추출한 것
    val_result = orig_targets.copy()
    val_result.loc[:, :] = 0

    # 
    sub_result = result_template.copy()
    sub_result.loc[:, 1:] = 0
    
    
    # 시드를 여러개 리스트업 해놓고 과적합 막기 위한 장치로 세팅??
    for h, seed in enumerate(seeds):

        seed_everything(seed)
        
        # 예측해야하는 output자체가 multilable classification이므로 
        # multilabelstratifiedkfold를 사용
        for i, (train_idx, valid_idx) in enumerate(MultilabelStratifiedKFold(n_splits=splits, random_state=seed, shuffle=True)
                                        .split(_y_i_train, _y_i_train)):
            print(f'Fold {i+1}')
            # array 형태로 뎅터 형식을 맞춰야 함
            if is_list:
                _X_train = [_X_i_train[0].loc[:,:].values[train_idx], _X_i_train[1].loc[:,:].values[train_idx]]
                _X_valid = [_X_i_train[0].loc[:,:].values[valid_idx], _X_i_train[1].loc[:,:].values[valid_idx]]
            else:
        
                _X_train = _X_i_train.loc[:,:].values[train_idx]
                _X_valid = _X_i_train.loc[:,:].values[valid_idx]
            
            # 예측 타겟 데이터도 배열 형태
            _y_train = _y_i_train.values[train_idx]
            _y_valid = _y_i_train.values[valid_idx]
            
            
            # 모델에 넣을 인풋 사이즈(변수의 갯수를 inPuttkdlwmfh sjgdma)
            if is_list:
                model = model_func(len(_X_i_train[0].columns), len(_X_i_train[1].columns))
            else:
                model = model_func(len(_X_i_train.columns))
            
            # NN 모델 학습 -> 학습에 필요한 파라미터에 대해서 공부가 필요
            model.fit(_X_train, _y_train,
                    validation_data=(_X_valid, _y_valid),
                    epochs=epochs, batch_size=batch_size,
                    callbacks=[
                        ReduceLROnPlateau(monitor='val_logloss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min')
                        , ModelCheckpoint(f'{name}_{seed}_{i}.hdf5', monitor = 'val_logloss', verbose = 0, save_best_only = True, save_weights_only = True, mode = 'min')
                        , EarlyStopping(monitor = 'val_logloss', min_delta = 1e-4, patience = 5, mode = 'min', baseline = None, restore_best_weights = True)
                    ], verbose=2)
            
            # 학습된 모델의 가중치를 적재
            model.load_weights(f'{name}_{seed}_{i}.hdf5')
            val_result.iloc[_y_i_train.iloc[valid_idx,:].index.values, :] += model.predict(_X_valid)

            if is_list:
                sub_result.loc[test_trt_index, sub_result.columns[1:]] += model.predict([_X_i_test[0].loc[test_trt_index, :], _X_i_test[1].loc[test_trt_index, :]])
            else:
                sub_result.loc[test_trt_index, sub_result.columns[1:]] += model.predict(_X_i_test.loc[test_trt_index, :])

            print('')

        tmp_result = val_result.copy()
        tmp_result.iloc[:,1:] = val_result.iloc[:,1:] / (h + 1)
        print(f' ---- seed:{seed}, ensemble:{h + 1}')
        if do_show_metrics:
            _ = show_metrics(tmp_result, show_each=False)

    val_result.iloc[:,1:] = val_result.iloc[:,1:] / len(seeds)
    if do_show_metrics:
        metrics = show_metrics(val_result, show_each=show_each_metrics)
    else:
        metrics = None

    sub_result.iloc[:, 1:] = sub_result.iloc[:, 1:] / (len(seeds) * splits)

    print(f' elapsed: {time.time() - st}')

    return sub_result, val_result, metrics

In [ ]:
# non_scored_target_cols: non_scored 데이터에서 가져온 컬럼(선택사항 컬럼) - validation셋인가?
# train_trt_index: 특정 범주형 데이터의 변수를 제외한 예측하고자하는 인덱스들
train_and_predict(
    'ADD', 
    create_add_model, 
    X_train, 
    y_non_train,
    X_test,
    train_targets_nonscored[non_scored_target_cols].loc[train_trt_index,:],
    non_scored_template,
    ADD_SEEDS, 
    ADD_SPLITS, 
    ADD_EPOCHS, 
    ADD_BATCH_SIZE, 
    shuffle_rows=False, 
    do_show_metrics=False,
    show_each_metrics=False
)

# Make Extra Features from non-scored targets
- To use non scored targets, I made a model and created non scored targets for test set.
- option 요소로 주어진 데이터를 활용하기 위해서 non-scored targets 모델을 생성함

In [31]:
p_min = 0.001
p_max = 0.999

# logloss 값을 학습시마다 지속적으로 반환
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))

In [32]:
# 어떤 역할을 하는 함수? bias_initializer
output_bias=tf.keras.initializers.Constant(-np.log(y_train.mean(axis=0).to_numpy()))

In [33]:
def create_add_model(input_dim):
    print(f'the input dim is {input_dim}')

    # M: keras 모델 - sequential 모델을 초기값으로 할당
    model = M.Sequential()
    # 인풋레이어 사이즈 정의
    model.add(L.Input(input_dim))
    # 레이어 Batchnormalization: 학습안정화 및 성능 개선
    # 신경망 모델의 인풋 데이터를 평균0, 분산1인 표준정규분포 상태로 만들어서 네트워크 학습이 잘 진행되도록 돕는 방식
    model.add(L.BatchNormalization())
    # regularization 효과를 창출 / 레이어의 모든 노드에서 나가는 activation을 특정 확률로 지움
    # 즉, 모델이 특정 환경에서 반복적으로 학습을 해서 도출하는 결과를 과하게 학습하지 않도록 만드는 것을 말함
    model.add(L.Dropout(0.5))
    # elu: relu
    model.add(WeightNormalization(L.Dense(input_dim, activation='elu')))

    model.add(L.BatchNormalization())
    model.add(L.Dropout(0.4))
    model.add(WeightNormalization(L.Dense(512, activation='selu')))
    
    model.add(L.BatchNormalization())
    model.add(L.Dropout(0.5))
    # output layer, 가중치 정규화 방식 사용
    model.add(WeightNormalization(L.Dense(y_non_train.shape[1], activation='sigmoid',
                                          bias_initializer=tf.keras.initializers.Constant(-np.log(y_non_train.mean(axis=0).to_numpy())
                                          ))))
    
    # model을 compile하는 이유는?
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(lr=1e-3), sync_period=10),
                loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=LABEL_SMOOTHING), metrics=logloss)

    return model

In [34]:
# non-scored target 데이터를 통한 예측 templates 생성
non_scored_template = sample_submission.copy()
non_scored_template = pd.DataFrame(non_scored_template.pop('sig_id'))
for col in non_scored_target_cols:
    non_scored_template[col] = 0.

In [224]:
for i, (train_idx, valid_idx) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=120, shuffle=True)
                                        .split(y_non_train, y_non_train)):
            print(f'Fold {i+1}')
        
        
    _X_train = X_train.loc[:,:].values[train_idx]
    _X_valid = X_train.loc[:,:].values[valid_idx]

    # 예측 타겟 데이터도 배열 형태
    _y_train = y_non_train.values[train_idx]
    _y_valid = y_non_train.values[valid_idx]

    model = create_add_model(len(X_train.columns))
    
    

the input dim is 947


In [35]:
model= create_add_model(len(X_train.columns))

the input dim is 947


In [233]:
len(X_train)*0.8

17558.4

In [36]:
train_idx = range(0,17558)
valid_idx = range(17558,len(X_train))

In [37]:
# NN 모델 학습 -> 학습에 필요한 파라미터에 대해서 공부가 필요
name = 'ADD'
seeds = 120
i = 0
model.fit(X_train.loc[:,:].values[train_idx], y_non_train.loc[:,:].values[train_idx],
        validation_data=(X_train.loc[:,:].values[train_idx], y_non_train.loc[:,:].values[train_idx]),
        epochs=60, batch_size=64,
        callbacks=[
            ReduceLROnPlateau(monitor='val_logloss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min')
            , ModelCheckpoint(f'{name}_{seeds}_{i}.hdf5', monitor = 'val_logloss', verbose = 0, save_best_only = True, save_weights_only = True, mode = 'min')
            , EarlyStopping(monitor = 'val_logloss', min_delta = 1e-4, patience = 5, mode = 'min', baseline = None, restore_best_weights = True)
        ], verbose=2)


Epoch 1/60
275/275 - 10s - loss: 0.0216 - logloss: 0.0193 - val_loss: 0.0144 - val_logloss: 0.0120
Epoch 2/60
275/275 - 9s - loss: 0.0126 - logloss: 0.0097 - val_loss: 0.0107 - val_logloss: 0.0077
Epoch 3/60
275/275 - 9s - loss: 0.0110 - logloss: 0.0078 - val_loss: 0.0103 - val_logloss: 0.0070
Epoch 4/60
275/275 - 9s - loss: 0.0106 - logloss: 0.0074 - val_loss: 0.0099 - val_logloss: 0.0066
Epoch 5/60
275/275 - 9s - loss: 0.0104 - logloss: 0.0071 - val_loss: 0.0095 - val_logloss: 0.0063
Epoch 6/60
275/275 - 9s - loss: 0.0101 - logloss: 0.0069 - val_loss: 0.0092 - val_logloss: 0.0060
Epoch 7/60
275/275 - 9s - loss: 0.0099 - logloss: 0.0067 - val_loss: 0.0090 - val_logloss: 0.0057
Epoch 8/60
275/275 - 9s - loss: 0.0097 - logloss: 0.0065 - val_loss: 0.0087 - val_logloss: 0.0055
Epoch 9/60
275/275 - 9s - loss: 0.0096 - logloss: 0.0063 - val_loss: 0.0085 - val_logloss: 0.0053
Epoch 10/60
275/275 - 9s - loss: 0.0095 - logloss: 0.0062 - val_loss: 0.0083 - val_logloss: 0.0051
Epoch 11/60
275/27

In [41]:
# 예측 테스
model.predict_proba(X_test)[0]

194

In [39]:
X_train

,g-0,g-2,g-3,g-4,g-5,g-7,g-8,g-9,g-10,g-11,...,gce_std,gce_skew,gce_kurt,gce_median,ge_sum,ge_mean,ge_std,ge_skew,ge_kurt,ge_median
index,,,,,,,,,,,,,,,,,,,,,
0,1.123421,-0.433396,-0.964591,-0.282548,-1.016012,-0.045026,0.698549,-0.307208,1.545668,0.167332,...,-0.231509,-1.382206,0.332903,0.391092,-0.466548,-0.466548,-0.200071,-1.323474,0.279847,-0.599367
1,0.118038,0.262404,0.096686,1.204425,0.695217,0.558623,-0.474493,0.840109,-1.250344,-0.585397,...,-0.311570,-0.121494,0.771762,0.957949,-0.069741,-0.069741,-0.230676,-0.019631,0.671087,0.448958
2,0.774613,1.417783,-0.114154,-0.020037,1.495514,0.358201,0.044292,1.247845,-0.655505,-0.777878,...,0.729941,0.040962,0.375723,-0.627145,-0.682173,-0.682173,0.757799,-0.153181,0.254737,-0.724672
3,-0.744187,-0.456376,0.772458,2.448425,-0.858771,0.298735,-0.139522,-1.371561,-1.013221,-0.505054,...,1.102178,-0.094962,-1.108978,-1.260776,-0.239230,-0.239230,1.017870,0.557489,-1.011274,-0.109471
4,-0.457396,0.962230,0.981547,1.454602,-0.869376,-0.230595,-0.981388,0.850767,-0.346712,-0.711659,...,0.717398,-2.232172,1.788667,0.710104,-0.933620,-0.933620,0.757162,-2.147718,1.589138,-0.000033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23808,0.236639,0.207837,-0.350717,-0.361947,0.575774,-0.276129,-0.860194,-0.022794,0.705877,-0.706175,...,-3.097103,-0.010938,-1.357223,0.846992,1.399084,1.399084,-2.973969,-0.201149,-1.318300,1.022804
23809,0.206721,-0.253901,-0.789493,-0.718753,0.923262,0.508876,-0.554977,-0.033930,0.286984,0.192944,...,-0.112217,-2.063024,1.858930,0.629524,0.068775,0.068775,-0.050632,-2.035350,1.766637,0.374497
23810,-1.940733,-0.603128,1.308103,-1.058194,0.856762,-0.736422,0.651861,0.648623,-0.303515,1.757062,...,-0.216652,-0.060783,-0.178504,-0.345776,0.067837,0.067837,-0.242659,-0.186023,-0.149545,-0.529686
